In [71]:
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy.io as sio

In [55]:
p_step = 3
w_size = 2 * p_step + 1

# Create grid of coordinates relative to center
y, x = np.mgrid[-p_step:p_step+1, -p_step:p_step+1]

# Create mask with border set to 0
center_mask = np.ones((w_size, w_size), dtype=bool)
center_mask[0, :] = False
center_mask[-1, :] = False
center_mask[:, 0] = False
center_mask[:, -1] = False

# Compute angles, shift by pi, convert to degrees
vectormatrix = x + 1j * y
angle_deg = np.degrees(np.angle(vectormatrix) + np.pi)
angle_deg = np.round(angle_deg)

# Function to zero out specified angles in the inner region
def zero_angles(angles, center_mask, target_angles):
    for t in target_angles:
        angles[(angles == t) & center_mask] = 0
    return angles

# Target angles to zero
targets = [0, 45, 90, 135, 180, 225, 270, 315, 360]

# Remove angles where inner mask is True
angle_deg = zero_angles(angle_deg, center_mask, targets)
angle_deg_mirr = np.flipud(angle_deg) * -1

# Get unique angles
unique_angles = np.unique(angle_deg).astype(np.int32)
unique_angles_mirr = np.unique(angle_deg_mirr).astype(np.int32)

imgs = [cv2.imread(file)[...,::-1] for file in glob("Input/RGB/*")]
cnts = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in glob("Input/SCM/*")]

for img, cont in zip(imgs, cnts):
    img = np.flip(img, axis=1).astype(np.float32) * 256
    cont = np.flip(cont, axis=1).astype(np.float32)

    pad_width = [(50, 50), (50, 50)]
    img = np.pad(img, pad_width + [(0, 0)], mode='constant', constant_values=0)
    cont = np.pad(cont, pad_width, mode='constant', constant_values=0)

In [70]:
state = sio.loadmat("Models/net-epoch-559.mat")["state"]
for x in state: print(x[0][0])

[[array([[[[-22.096626  ,  40.85179   , -16.005777  , ...,  -2.0684216 ,
             16.243233  ,  11.408001  ],
           [-17.495186  ,  36.02282   , -16.510275  , ...,  -1.95877   ,
             12.229503  ,   8.790751  ],
           [-18.414724  ,  33.984844  , -20.236425  , ...,   1.643784  ,
             10.929113  ,  12.992814  ],
           [-13.979246  ,   1.1668022 ,  13.212273  , ..., -35.248554  ,
            -12.478444  ,  19.060059  ]],

          [[-20.210625  ,  38.35376   , -21.429344  , ...,   4.494857  ,
             12.425456  ,   3.269052  ],
           [-16.415325  ,  33.3218    , -20.75902   , ...,   4.0335374 ,
             10.283826  ,   0.7623112 ],
           [-15.081586  ,  31.080114  , -22.73043   , ...,   6.159605  ,
              7.9391623 ,   7.6088505 ],
           [ -9.70496   ,   2.3324685 ,   7.544709  , ..., -26.267235  ,
            -18.949886  ,  21.247925  ]],

          [[-11.466738  ,  37.8719    , -14.476059  , ...,   0.13478279,
           

In [ ]:
# Use MSE loss for training
class TracerNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, ceil_mode=True)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, padding=0),
        )
        
        self.fc = nn.Linear(1024, 1)
    
    def load_mat_state(self, file):
        params = sio.loadmat(file)['state'][0][0][0][0]
        params = [torch.tensor(x.transpose(x.ndim - 1 - np.arange(x.ndim))).squeeze() for x in params]

        def load_linear(layer, start_idx):
            layer.weight = nn.Parameter(params[start_idx])
            layer.bias = nn.Parameter(params[start_idx+1])

        def load_conv_block(block, start_idx):
            load_linear(block[0], start_idx)
            if len(block) > 2: load_linear(block[1], start_idx+2)

        # can stay hardcoded for now
        load_conv_block(self.conv1, 0)
        load_conv_block(self.conv2, 5)
        load_conv_block(self.conv3, 10)
        load_conv_block(self.conv4, 15)
        load_conv_block(self.conv5, 20)
        load_linear(self.fc, 22)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.fc(x.flatten(1))
        return x
    
tracer_net = TracerNet()
tracer_net.load_mat_state("Models/net-epoch-559.mat")
tracer_net.to("cuda")